## not completed yet

In [1]:
import pandas as pd
import ast
import os
import numpy as np
import sys 

In [5]:
region = 'VISp'
path_table_info = f'/crnldata/cophy/TeamProjects/mohammad/ibl-oscillations/_analyses/extraction_module/data/eid_probe_info_{region}_{region}.csv'
table = pd.read_csv(path_table_info)
table['pid1'] = table['pid1'].apply(ast.literal_eval)
pids = []
eids = []
for i, row in enumerate(table.iterrows()):
    eid = row[1]['eid']
    pid1 = row[1]['pid1']
    for pid in pid1:
        if os.path.isfile(f'/mnt/data/AdaptiveControl/IBLrawdata/pid_data/{pid}/lfp_{pid}_raw.fif'):
            pids.append(pid)
            eids.append(eid)
print(f'Number of pids with files: {len(pids)} for region {region}')

Number of pids with files: 46 for region VISp


In [10]:

from compute_decodability import  Right_left_decodability_SpikesOnDepths
import submitit 
pids_short = pids[:]
eids_short = eids[:]
executor = submitit.AutoExecutor(folder=os.getcwd()+'/logs/')
executor.update_parameters(mem_gb=60, timeout_min=500, slurm_partition="CPU", cpus_per_task=1)
job = executor.submit(Right_left_decodability_SpikesOnDepths, pids, eids, average_period = [0.1, 1], output_path = 'data/decodability_results.pkl', t_bin=0.1, d_bin=20, pre_stim=0.4, post_stim=1, depth_lim=[0, 3840])

        

In [3]:
import pickle
import pandas as pd
import numpy as np

# Load the results from the pickle file
with open('V1_cluster_decodability.pkl', 'rb') as pickle_file:
    results = pickle.load(pickle_file)

# # select the first pid
# pid = list(results.keys())[0]
# # Extract unique clusters
# unique_clusters = results[pid]['unique_clusters']

# Initialize lists for layers and ROC values
layers_list = []
ROC_list = []


# Extract layers and ROC values from the loaded results
for pid, data in results.items():
    layers_list.extend(data['layers'])  # Append layers
    ROC_list.extend(data['ROC'])        # Append ROC values

# Convert lists to pandas Series and numpy array
layers_concat = pd.Series(layers_list)
ROC_concat = np.array(ROC_list)
ROC_concat = np.abs(ROC_concat - 0.5)  # Adjust ROC values

# Define valid regions
valid_regions = ['VISp1', 'VISp2/3', 'VISp4', 'VISp5', 'VISp6a', 'VISp6b']

# Print mean AUC-ROC and number of clusters for each valid region
for layer in valid_regions:
    layer_idx = layers_concat.str.contains(layer)
    layer_ROC = ROC_concat[layer_idx]
    
    if layer_ROC.size > 0:  # Ensure there are clusters for the layer
        print(f'Layer: {layer}, Mean AUC-ROC: {layer_ROC.mean():.4f}, max: {layer_ROC.max()}  Num Clusters: {layer_ROC.size}')
    else:
        print(f'Layer: {layer}, No valid clusters found.')

Layer: VISp1, Mean AUC-ROC: 0.3168, max: 0.5 min Num Clusters: 88
Layer: VISp2/3, Mean AUC-ROC: 0.2803, max: 0.5 min Num Clusters: 648
Layer: VISp4, Mean AUC-ROC: 0.2196, max: 0.5 min Num Clusters: 615
Layer: VISp5, Mean AUC-ROC: 0.2393, max: 0.5 min Num Clusters: 1296
Layer: VISp6a, Mean AUC-ROC: 0.2406, max: 0.5 min Num Clusters: 993
Layer: VISp6b, Mean AUC-ROC: 0.2176, max: 0.5 min Num Clusters: 142


In [5]:
# Initialize lists for layers and ROC values
layers_list = []
ROC_list = []

# Extract layers and ROC values from the loaded results
for pid, data in results.items():
    layers_list.extend(data['layers'])  # Append layers
    ROC_list.extend(data['ROC'])        # Append ROC values

# Convert lists to pandas Series and numpy array
layers_concat = pd.Series(layers_list)
ROC_concat = np.array(ROC_list)

# Define valid regions
valid_regions = ['VISp1', 'VISp2/3', 'VISp4', 'VISp5', 'VISp6a', 'VISp6b']

# Print significant ROC statistics for each valid region
for layer in valid_regions:
    layer_idx = layers_concat.str.contains(layer)
    layer_ROC = ROC_concat[layer_idx]
    
    if layer_ROC.size > 0:  # Ensure there are clusters for the layer
        # Count total number of clusters in the layer
        total_clusters = layer_ROC.size
        
        # Select significant ROC values (less than 0.3 or greater than 0.7)
        signif_ROC = layer_ROC[(layer_ROC < 0.3) | (layer_ROC > 0.7)]
        
        # Calculate percentage of significant clusters
        percent_signif = (signif_ROC.size / total_clusters) * 100
        
        # Subtract 0.5 from significant ROC values and take the absolute value
        adjusted_signif_ROC = np.abs(signif_ROC - 0.5)
        
        # Compute the mean of the adjusted significant ROC values
        if signif_ROC.size > 0:  # Ensure there are significant clusters
            mean_adjusted_ROC = adjusted_signif_ROC.mean()
            print(f'Layer: {layer}, Percent Significant: {percent_signif:.2f}%, Mean Adjusted AUC-ROC: {mean_adjusted_ROC:.4f}, Num Significant Clusters: {signif_ROC.size}')
        else:
            print(f'Layer: {layer}, No significant clusters found.')
    else:
        print(f'Layer: {layer}, No valid clusters found.')


Layer: VISp1, Percent Significant: 70.45%, Mean Adjusted AUC-ROC: 0.4106, Num Significant Clusters: 62
Layer: VISp2/3, Percent Significant: 64.20%, Mean Adjusted AUC-ROC: 0.4001, Num Significant Clusters: 416
Layer: VISp4, Percent Significant: 47.48%, Mean Adjusted AUC-ROC: 0.3746, Num Significant Clusters: 292
Layer: VISp5, Percent Significant: 52.62%, Mean Adjusted AUC-ROC: 0.3758, Num Significant Clusters: 682
Layer: VISp6a, Percent Significant: 54.08%, Mean Adjusted AUC-ROC: 0.3803, Num Significant Clusters: 537
Layer: VISp6b, Percent Significant: 50.00%, Mean Adjusted AUC-ROC: 0.3721, Num Significant Clusters: 71
